## DESE61003 - Audio Experience Design
# Week 5 - Knowledge-Based Sound Recognition

In [ ]:
from disiple.signals import AudioSignal, Spectrum, PowerSpectrum, Spectrogram, TimeSignal
from disiple.util import download_and_extract_archive, download_file
import numpy as np
from scipy import signal

In this session, we'll get started with recognising different sounds. To do so, we will learn how to access the underlying data in `disiple` objects, such that we can add custom functionality using [`NumPy`](https://numpy.org/doc/stable/) and [`SciPy`](https://scipy.github.io/devdocs/index.html) functions.

### 1. Loading audio files

This session will become a lot more interesting if we don't need to stick to the few example audio files, but can load our own files. When running this notebook on MyBinder though, it is not trivial to do so, since we're working on a computer in the cloud and cannot access its hard disk through the operating system. Therefore the `download_and_extract_archive()` function (already loaded from `disiple.util` above) will come in handy. Even when running this notebook locally, automating the audio download is useful, since it's effortless and reproducible, for instance when moving the notebook to another computer.

As the name `download_and_extract_archive()` implies, we need an archive with audio files to download from somewhere. The [University of Iowa Musical Instrument Samples](https://theremin.music.uiowa.edu/MIS.html) is a good source for samples of musical instruments. Browse around and see if there's anything that interests you, otherwise you can look further on the web.

For the sake of this example, we're going to download a zip of flute recordings. Then we just need to decide where to unzip them. All audio files we've used so far were stored in the `../data` folder, so let's follow this example and create a subfolder there.

In [ ]:
flute_url = 'https://theremin.music.uiowa.edu/sound%20files/MIS%20Pitches%20-%202014/Woodwinds/Flute/Flute.nonvib.ff.stereo.zip'
flute_folder = '../data/flute'
download_and_extract_archive(flute_url, flute_folder, verbose=True)

We can check the filesystem to see if the downloading worked by using the `%ls` command in a cell. This is a so-called [_magic command_](https://ipython.readthedocs.io/en/stable/interactive/magics.html), which comes built-in with Jupyter notebooks (so it's not a Python command). Like the `ls` Unix command, it shows contents of the given folder on disk.

In [ ]:
%ls ../data

We can see some example audio files in the `data` folder, as well as the newly created `flute` folder.

In [ ]:
%ls ../data/flute

And inside the folder we can see a number of files in [AIFF format](https://en.wikipedia.org/wiki/Audio_Interchange_File_Format), each containing a different note.

If you'd rather download individual files from the internet, you can use the similar function `download_file()`.

In [ ]:
orch_url = 'https://upload.wikimedia.org/wikipedia/commons/7/7d/Orchestra_hit.ogg'
orch_folder = '../data'
download_file(orch_url, orch_folder, verbose=True)

Once you have an audio file on disk, you load it into an `AudioSignal` by passing its paths as the first argument. Multi-channel audio will always be mixed down to mono.

In [ ]:
orchestra_hit = AudioSignal('../data/Orchestra_hit.ogg')
orchestra_hit.play()
orchestra_hit.samplerate

By default, an `AudioSignal` gets created with the same sample rate as the file on disk. You can inspect this by checking the `samplerate` property of the `AudioSignal`. However, you can also request resampling to a particular rate at the opening of the file by passing the additional `samplerate=` argument. This has exactly the same result as calling the `.resample()` method on the `AudioSignal` afterwards, but saves typing and the creation of the intermediary signal. Resampling at read can be useful to bring multiple signals of different origins onto the same common sample rate.

In [ ]:
resampled_orchestra_hit = AudioSignal('../data/Orchestra_hit.ogg', samplerate=4000)
resampled_orchestra_hit.play()
resampled_orchestra_hit.samplerate

### 2. Getting the underlying data out of `disiple` objects

The aim of the `disiple` library is to make it quick and easy to plot and play back signals in their most common representations. It is used to hide boilerplate code from these notebooks and provide sensible defaults to get you started. It is designed to be a light-weight wrapper around much larger libraries, meaning that it should be easy to access the underlying data such that extra functionality can be added without needing to add it to the `disiple` library itself.

Concretely, all data in each of the three classes you've used so far (`AudioSignal`, `(Power)Spectrum` and `Spectrogram`) is stored as [NumPy arrays](https://numpy.org/doc/stable/user/quickstart.html). Any function you're going to write next will therefore take NumPy arrays as input. This also means you'll be able to use these functions without using `disiple`, for instance with audio you've read through other libraries or signals created directly with NumPy.

The only thing left is to show you how to access the NumPy arrays in each of the `disiple` classes. For an `AudioSignal`, the samples can be accessed through the `.samples` property. This way you get a variable of type `numpy.ndarray` which you can use like any other. For instance, you can [index](https://numpy.org/doc/stable/user/basics.indexing.html) it with the `[]` operator or get its [shape](https://numpy.org/doc/stable/reference/generated/numpy.shape.html).

In [ ]:
type(orchestra_hit.samples), orchestra_hit.samples[:5], orchestra_hit.samples.shape

Together with the above-mentioned `.samplerate` property, you have all info you need to process the signal in the time domain. For convenience, you can also get an array containing the time associated with each sample through the `.times` property.

In [ ]:
orchestra_hit.samplerate, orchestra_hit.times[:5]

Probably the most useful data in a `Spectrum` can be accessed through its `.magnitude` property.

In [ ]:
orch_spectrum = Spectrum(orchestra_hit)
orch_spectrum.magnitude[:5]

This gives you the spectral values as calculated according to the given options, so in case of a `PowerSpectrum` or when using decibel units, this will actually return the squared magnitude values (i.e. power). By default, a linear spectrum in decibels gets computed, without single side-band normalisation. If you need help remembering what options are accepted by the `Spectrum` constructor, you can check its [_type signature_](https://en.wikipedia.org/wiki/Type_signature) by appending `?` to the class name, as below.

In [ ]:
Spectrum?

For instance, the snippet below returns the magnitude in linear units of a logarithmic spectrum between 110 and 880 Hz with 3 bins per semitone and single side-band normalisation.

In [ ]:
Spectrum(orchestra_hit, dB=False, spacing='log', min_freq=110, max_freq=880, num_bins=3, norm_single_side_band=True).magnitude[:5]

You can further access the phase values or the complex spectral values with the `.phase` and `.complex` properties, respectively, although these will have less practical use for the remainder of the module. Finally, the frequencies of the bins can be found in `.frequencies`.

In [ ]:
orch_spectrum.phase[:5], orch_spectrum.complex[:5], orch_spectrum.frequencies[:5]

The underlying data in a `Spectrogram` is two dimensional, with frequency bins in the rows and time stamps in the columns. Similarly to `Spectrum`, we can again access magnitude (or power), phase or complex spectrogram values with the `.magnitude`, `.phase` and `.complex` properties.

In [ ]:
orch_specgram = Spectrogram(orchestra_hit, frame_size=2048)
orch_specgram.magnitude.shape, orch_specgram.phase.shape, orch_specgram.complex.shape

The values of the frequency bins and time stamps themselves can again be obtained through the `.frequencies` and `.times` properties. Together they form the grid on which the spectrogram values lie.

In [ ]:
len(orch_specgram.frequencies), len(orch_specgram.times)

### 3. Implementing an audio feature

Hopefully playing around with Sonic Visualiser and the libxtract Vamp Plugins has given you an idea about what type of feature will be most useful to recognise the signals you're interested in. Check [Peeters'report](http://recherche.ircam.fr/anasyn/peeters/ARTICLES/Peeters_2003_cuidadoaudiofeatures.pdf) to find out how to implement it or for futher inspiration. You might even skip reading about existing features and try coming up with one yourself, based on some [common statistics](https://numpy.org/doc/stable/reference/routines.statistics.html) applied to one or more signal representations (there's a high chance someone will have already done something similar, but don't let that spoil the fun). If that's the case, skip ahead and go for it. If not, keep following along for an example where we recreate the spectral centroid example shown in Sonic Visualiser.

To start, we'll load the example trumpet and flute signal.

In [ ]:
trumpet = AudioSignal('../data/Trumpet.novib.ff.A4.mono.wav', samplerate=8000)
flute = AudioSignal('../data/Flute.nonvib.ff.A4.mono.wav', samplerate=8000)

We can see that the distribution of power over their harmonic components is very different. The harmonics of the flute are almost equally powerful up to the third and rapidly decrease afterwards, whereas the power in the trumpet's harmonics increases up to the third harmonic before it starts decreasing gradually. Note that we use linear units for the spectrum here, instead of decibels, which is the normal starting point for calculating spectral features. We also ensure that both signals have the same frequency resolution by requesting the number of frequency bins to be equal to the longest of the two signals.

In [ ]:
num_bins = max(len(trumpet), len(flute))
trumpet_spectrum = Spectrum(trumpet, dB=False, num_bins=num_bins)
trumpet_spectrum.magnitude /= trumpet_spectrum.magnitude.sum(axis=0, keepdims=True)
trumpet_spectrum.display(title='Spectrum of a trumpet')
flute_spectrum = Spectrum(flute, dB=False, num_bins=num_bins)
flute_spectrum.magnitude /= flute_spectrum.magnitude.sum(axis=0, keepdims=True)
flute_spectrum.display(title='Spectrum of a flute')

We are going to capture this difference more explicitly by turning it into a one-dimensional signal, through the calculation of the spectral centroid: $$\mathrm{centroid}(m) = \dfrac{\sum_k k \left|X_{k,m}\right|}{\sum_k \left|X_{k,m}\right|}$$ where $\left|X_{k,m}\right|$ is the spectral magnitude value for frequency index $k$ and time index $m$. Because the magnitude value $\left|X_{k,m}\right|$ scales with the power of the frame $k$, we usually calculate the spectral centroid with the normalised magnitude values $|\tilde{X}_{k,m}|$: $$|\tilde{X}_{k,m}| = \dfrac{|X_{k,m}|}{\sum_k |X_{k,m}|}$$

#### Activity 1

Complete the function below such that it returns the spectral centroids for the magnitude values given as a NumPy array. Include both the magnitude normalisation step and the actual centroid calculation. There are many ways to implement this, but you'll find the [`np.sum()`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) function or the equivalent [`np.ndarray.sum()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.sum.html) method certainly useful. Make sure you understand the use of the `axis=` and `keepdims=` arguments. You should also be aware of the difference between the [`np.multiply`](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html) _element-wise (or array)_ product (for which the `*` operator is a syntactic shortcut) and the [`np.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) _matrix_ product (for which the `@` operator is a syntactic shortcut). Finally, knowing about [`np.mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html), [`np.average`](https://numpy.org/doc/stable/reference/generated/numpy.average.html) and the difference between them will allow you to write compact code. There is also a computational optimisation you can apply to the combination of magnitude normalisation and spectral centroid calculation. Can you spot what it is?

In [ ]:
def spectral_centroid(magnitudes):
    # replace the placeholder code below with your implementation
    spectral_centroids = np.zeros(magnitudes.shape[1])
    return spectral_centroids

If our different signals are nicely cut into separate files, we can simply calculate the spectral centroid for the spectrum of each file (considering the spectrum as a special case of a spectrogram with just 1 time step).

In [ ]:
spectral_centroid(trumpet_spectrum.magnitude), spectral_centroid(flute_spectrum.magnitude)

The resulting values allow us to distinguish between these two signals playing the same note on a different instrument. Because the trumpet has more power in its higher harmonics, the centre of mass of the spectrum (i.e. the spectral centroid) will be higher for the trumpet than for the flute.

In case we want to identify where in a signal each instrument appears, we calculate the spectral centroid with the magnitudes of a spectrogram (again making sure to use linear units, not decibels).

In [ ]:
trumpet_flute = trumpet & flute
trumpet_flute_specgram = Spectrogram(trumpet_flute, frame_size=1024, step_size=1024, dB=False)
trumpet_flute_specgram.display(title='Spectrogram of a trumpet followed by a flute')

The result is a new signal, which gives a one-dimensional value (the spectral centroid) for each frame.

In [ ]:
spectral_centroid(trumpet_flute_specgram.magnitude).shape

Since we know this new signal has the same sample rate as the spectrogram, we can display it using a `disiple` class we haven't used up to now. A `TimeSignal` is the more generic version of an `AudioSignal`, which does not require the amplitude to be contrained within the interval [-1, 1] (and cannot be played back as audio). You create one by passing its samples and sample rate to the constructor, then you can use a similar `.display()` method as for all `disiple` objects.

In [ ]:
centroid_samples = spectral_centroid(trumpet_flute_specgram.magnitude)
centroid_signal = TimeSignal(centroid_samples, trumpet_flute_specgram.samplerate)
centroid_signal.display(y_axis_label='spectral centroid', y_range=(0, 0.002), title='Spectral centroid for a signal of a trumpet followed by a flute')
trumpet_flute.display()

The centroid signal clearly has different values for frames containig the trumpet than for those containing the flute, but because of the normalisation we applied to the spectral values, the silent parts strongly disrupt the centroid signal. To improve the clarity of the centroid signal, we'd like to remove the near-silent frames from the centroid calculation (or more practically, override the spectral centroid values for near-silent frames).

Just like variables with a data type `AudioSignal` and `Spectrum`, `Spectrogram` variables also have a method `.power()` that returns the power in decibels (by default) or in linear units (when passing the argument `dB=False`). Unlike the former two, the result is not a scalar but a NumPy array containing the power _for each frame_.

In [ ]:
frames_power = trumpet_flute_specgram.power(dB=True)
type(frames_power), frames_power.shape

This too can be considered a new `TimeSignal` with the same sample rate as the spectrogram, so we can visualise it in the same way as the spectral centroid.

#### Activity 2

Plot the power for each frame by creating a `TimeSignal` from the NumPy array of values.

In [ ]:
# your code here

From this plot, we can see that putting a threshold at -40 dB would be suitable to separate the near-silent frames from the frames actually containing an instrument.

#### Activity 3

Use either [boolean array indexing](https://numpy.org/doc/stable/user/basics.indexing.html#boolean-array-indexing) or the [`np.where`](https://numpy.org/doc/stable/reference/generated/numpy.where.html) function to create a cleaned version of the centroid samples where all spectral centroid values that are computed from near-silent frames are set to zero. Then plot the resulting signal.

In [ ]:
# your code here

The resulting cleaned up centroid signal can easily be turned into an instrument detection function by thresholding. For instance, all frames with a centroid value above 0.0014 can be considered as trumpet and everything below (but more than zero) as flute.

Now that you've completed this example, hopefully you feel prepared to implement the feature of your choice, but keep on reading for some more useful tips.

### 4. Frames in the time domain

If you're not exactly sure beforehand if and where a sound of interest is located in a longer signal (which can contain silence or other, distracting sounds), splitting a signal into frames is an essential step to localise its presence. If you want to work in the frequency domain, `Spectrogram` provides this functionality, but what to do if you want to stay in the time domain for further processing? Luckily an `AudioSignal` has a `.frames()` method which cuts the time signal up into shorter frames using the same arguments as for `Spectrogram`. So you can use `frame_size=` and `step_size=` to specify frame width and step size as a number of samples, or `frame_duration` and `step_duration` to specify them as durations, or a any combination thereof. The result is a list of shorter `AudioSignal`s, which you can use to listen to and display the individual frames.

In [ ]:
orch_frames = orchestra_hit.frames(frame_size=2048, step_size=2048)
orch_frame_9 = orch_frames[8]
orch_frame_9.play()
orch_frame_9.display(title='The ninth frame of the orchestra hit sound')

For further processing, this list of separate `AudioSignal`s is not very convenient though, as you'd need get the samples from each signal separately. Therefore a similar method `.frame_data()` is available, which takes the same arguments but return a two-dimensional NumPy array with the frames in the columns

In [ ]:
frame_data = orchestra_hit.frame_data(frame_size=2048, step_size=2048)
frame_data.shape

You can then get the temporal values of the ninth frame as:

In [ ]:
frame_data[:, 8]

#### Activity 4

Use the `frame_data` NumPy array to calculate the power per frame in the time domain. Verify your implementation by comparing it with the power per frame you get from calling the `.power(dB=False)` method on its `Spectrogram`. Note that calculating the `Spectrogram` by default includes applying a window to the frames, which changes their power. In order to disable this, and make calculating power in the temporal and spectral domain comparable, you need to pass `window='rect'` to the `Spectrogram` constructor.

In [ ]:
# your code here

### 5. Designing around constraints

Distinguishing between two arbitrary sounds, or recognising a particular sound in a longer signal is not necessarily easy. Luckily, there are plenty of applications where there is some flexibility around which particular sounds are chosen to be recognised. For instance, Amazon didn't _have_ to choose Alexa as the keyword to wake up its digital assistant, but doing so [helped to make its detection more robust because of the hard consonant X](https://www.nytimes.com/2017/01/16/opinion/ask-alexa-no-hear-this-alexa.html). Similarly, we can work backwards and see which sounds are easy to distinguish or recognis with a particular feature (a given constraint).

To get a good idea of what a specific feature is capable of, we need a wide variety of test signals to try it with. Therefore we start by loading some audio files and bringing them all onto a common sample rate.

In [ ]:
samplerate = 44100

In [ ]:
clarinet_a4 = AudioSignal('../data/EbClarinet.ff.A4.mono.wav', samplerate)
clarinet_c6 = AudioSignal('../data/EbClarinet.ff.C6.mono.wav', samplerate)
flute_a4 = AudioSignal('../data/Flute.nonvib.ff.A4.mono.wav', samplerate)
flute_c6 = AudioSignal('../data/Flute.nonvib.ff.C6.mono.wav', samplerate)
trumpet_a4 = AudioSignal('../data/Trumpet.novib.ff.A4.mono.wav', samplerate)
trumpet_c6 = AudioSignal('../data/Trumpet.novib.ff.C6.mono.wav', samplerate)
violin_a4 = AudioSignal('../data/Violin.arco.ff.sulG.A4.mono.wav', samplerate)
violin_c6 = AudioSignal('../data/Violin.arco.ff.sulD.C6.mono.wav', samplerate)
go = AudioSignal('../data/go.wav', samplerate)
go_lower = AudioSignal('../data/go-lower.wav', samplerate)
stop = AudioSignal('../data/stop.wav', samplerate)
stop_higher = AudioSignal('../data/stop-higher.wav', samplerate)

In order to systematically process all and report the results, it is convenient to store all signals in a dictionary.

In [ ]:
recorded_signals = {
    'clarinet A4': clarinet_a4,
    'clarinet C6': clarinet_c6,
    'flute A4': flute_a4,
    'flute C6': flute_c6,
    'trumpet A4': trumpet_a4,
    'trumpet C6': trumpet_c6,
    'violin A4': violin_a4,
    'violin C6': violin_c6,
    'go': go,
    'go_lower': go_lower,
    'stop': stop,
    'stop_higher': stop_higher,
}

Apart from having a number of real recorded signals, it is generally a good idea to have some synthetic signals. These are simpler and less noisy, which will help in debugging your algorithms.

#### Activity 5

Create some synthesised signals using the techniques we've seen in the first session and add them to the dictionary below.

In [ ]:
synthetic_signals = {}

We can then join both dictionaries into one larger, such that we can easily run over all our test signals.

In [ ]:
all_signals = {**synthetic_signals, **recorded_signals}
# note: from Python 3.9 on, you can do: all_signals = recorded_signals | synthetic_signals

For example, we can start by creating a player for each sound so we can quickly check what they sound like.

In [ ]:
for name, audio_signal in all_signals.items():
    print(name)
    audio_signal.play()

A good first feature to try is the [zero-crossing rate](https://en.wikipedia.org/wiki/Zero-crossing_rate). It is calculated in the time domain and relatively easy to implement.

#### Activity 6

Look up the definition of zero-crossing rate online or in [Peeter's report](http://recherche.ircam.fr/anasyn/peeters/ARTICLES/Peeters_2003_cuidadoaudiofeatures.pdf) and complete the function defined below, which takes an NumPy array of samples as its only argument. Test it first with synthetic signals, for which you can deduce the theoretical result, then with real signals. What do you notice?

In [ ]:
def zero_crossing_rate(samples):
    # replace the placeholder code below with your implementation
    return 0

In [ ]:
# your code here

_your answer here_

#### Activity 7

The zero-crossing rate can be used as a primitive form of pitch detection for monophonic signals. Can you figure out how to convert a zero-crossing rate into a fundamental frequency prediction? Do implement this conversion and run it for all signals. Does it give satisfactory results?

In [ ]:
# your code here

_your answer here_

### 6. Further ideas

Now that you've implemented at least a feature in the temporal and in the spectral domain, you can experiment some more. Can you identify the most promising property such as pitch or timbre to distinguish between any two given signals? If you want a further challenge, try identifying sounds based on their spectral envelope. The trick to know is that an envelope is essentially a low-frequency contour of the signal. We've seen some functionality that could help obtaining it in the first session, or you could have a look at [`signal.medfilt()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.medfilt.html). Have a browse through the [`scipy.signal`](https://docs.scipy.org/doc/scipy/reference/signal.html) package to find some more functions that can be useful. For instance, quite some features include a peak-picking step somewhere, which is provided by [`signal.find_peaks()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html).